Task#1

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
def sample_image(image, factor):
    h, w = image.shape[:2]
    return cv2.resize(image, (max(1, w // factor), max(1, h // factor)), interpolation=cv2.INTER_NEAREST)

def quantize_image(image, levels):
    step = max(1, 256 // levels)
    q = np.floor(image / step) * step
    return q.astype(np.uint8)

def plot_triplet(original, sampled, quantized, title_suffix=""):
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 3, 1)
    plt.imshow(original, cmap='gray')
    plt.title('Original' + title_suffix)
    plt.axis('off')
    plt.subplot(1, 3, 2)
    plt.imshow(sampled, cmap='gray')
    plt.title('Sampled' + title_suffix)
    plt.axis('off')
    plt.subplot(1, 3, 3)
    plt.imshow(quantized, cmap='gray')
    plt.title('Quantized' + title_suffix)
    plt.axis('off')
    plt.show()

In [ ]:
def load_gray(path, fallback_shape=(256, 256)):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        rng = np.random.default_rng(0)
        img = (rng.random(fallback_shape) * 255).astype(np.uint8)
    return img

In [ ]:
image_path = '../images/lena_gray_256.tif'
original = load_gray(image_path)

for sampling_factor, quantization_levels in [(2, 2), (4, 4), (8, 8), (14, 9), (16, 16)]:
    sampled = sample_image(original, sampling_factor)
    quantized = quantize_image(original, quantization_levels)
    plot_triplet(original, sampled, quantized, f' (S={sampling_factor}, Q={quantization_levels})')

Task#2

In [ ]:
def load_pil_gray(path, size=(400, 400)):
    try:
        img = Image.open(path).convert('L').resize(size, Image.Resampling.LANCZOS)
        return np.asarray(img, dtype=np.uint8)
    except Exception:
        rng = np.random.default_rng(1)
        return (rng.random(size[::-1]) * 255).astype(np.uint8)

im1 = load_pil_gray('../images/lena_gray_256.tif')
im2 = load_pil_gray('../images/cameraman.tif')

def show(img, title):
    plt.figure(figsize=(5, 5))
    plt.imshow(img, cmap='gray', vmin=0, vmax=255)
    plt.title(title)
    plt.axis('off')
    plt.show()

In [ ]:
subtracted = cv2.subtract(im1, im2)
show(subtracted, 'Subtract (im1 - im2)')

In [ ]:
added_constant = np.clip(im1.astype(np.int16) + 175, 0, 255).astype(np.uint8)
show(added_constant, 'Add Constant 175')

In [ ]:
a = load_pil_gray('../images/A.png')
b = load_pil_gray('../images/B.png')

set_difference = cv2.bitwise_and(a, cv2.bitwise_not(b))
symmetric_difference = cv2.bitwise_xor(a, b)
intersection = cv2.bitwise_and(a, b)

show(a, 'A')
show(b, 'B')
show(set_difference, 'Set Difference (A \ B)')
show(symmetric_difference, 'Symmetric Difference (A XOR B)')
show(intersection, 'Intersection (A AND B)')